In [1]:
# [hrs_studied, hrs_slept, pass/fail]
# pass = 1, fail = 0
# classify into pass or fail

data = [
    [1, 3, 0]
    [2, 4, 0],
    [3, 5, 0],
    [4, 5, 0],
    [5, 6, 1],
    [6, 6, 1],
    [7, 7, 1],
    [8, 6, 1],
]

### Implementation

In [19]:
def gini_index(data: list[list[int]]) -> float:
    if len(data) == 0:
        return 0.0

    total = len(data)
    count_0 = sum(1 for d in data if d[-1] == 0)
    count_1 = sum(1 for d in data if d[-1] == 1)
    p0 = count_0 / total
    p1 = count_1 / total
    return 1 - (p0 ** 2 + p1 ** 2)


def gain(gini_before: float, gini_after: float) -> float:
    return gini_before - gini_after

def split(data: list[list[int]]):
    best_gain = -1
    best_feature = None
    best_threshold = None

    gini_before = gini_index(data)
    n_total = len(data)
    num_features = len(data[0]) - 1

    for feature in range(num_features):
        values = sorted(set(d[feature] for d in data))
        thresholds = [(values[i] + values[i + 1]) / 2 for i in range(len(values) - 1)]
        
        for t in thresholds:
            left = [d for d in data if d[feature] <= t]
            right = [d for d in data if d[feature] > t]

            if len(left) == 0 or len(right) == 0:
                continue

            gini_left = gini_index(left)
            gini_right = gini_index(right)

            gini_after = (
                (len(left) / n_total) * gini_left
                + (len(right) / n_total) * gini_right
            )

            current_gain = gain(gini_before, gini_after)

            if current_gain > best_gain:
                best_gain = current_gain
                best_feature = feature
                best_threshold = t

    return best_feature, best_threshold, best_gain

def majority_class(data: list[list[int]]) -> int:
    count_0 = sum(1 for d in data if d[-1] == 0)
    count_1 = sum(1 for d in data if d[-1] == 1)
    return 0 if count_0 >= count_1 else 1

def is_pure(data: list[list[int]]) -> bool:
    labels = [d[-1] for d in data]
    return len(set(labels)) == 1

def build_tree(data: list[list[int]], depth=0, max_depth=3) -> dict:
    if is_pure(data):
        return data[0][-1]

    if depth == max_depth:
        return majority_class(data)

    feature, threshold, best_gain = split(data)

    if best_gain <= 0:
        return majority_class(data)

    left_data = [d for d in data if d[feature] <= threshold]
    right_data = [d for d in data if d[feature] > threshold]

    if len(left_data) == 0 or len(right_data) == 0:
        return majority_class(data)

    return {
        "feature": feature,
        "threshold": threshold,
        "left": build_tree(left_data, depth + 1, max_depth),
        "right": build_tree(right_data, depth + 1, max_depth)
    }

def predict(tree, sample):
    if isinstance(tree, int):
        return tree

    if sample[tree["feature"]] <= tree["threshold"]:
        return predict(tree["left"], sample)
    else:
        return predict(tree["right"], sample)

tree = build_tree(data)
for d in data:
    x = d[:-1]
    y = d[-1]
    print(f"Predicted: {predict(tree, x)} \nActual: {y}\n")

Predicted: 0 
Actual: 0

Predicted: 0 
Actual: 0

Predicted: 0 
Actual: 0

Predicted: 1 
Actual: 1

Predicted: 1 
Actual: 1

Predicted: 1 
Actual: 1

Predicted: 1 
Actual: 1

Predicted: 0 
Actual: 0



### With Scikit-Learn

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X = [d[:-1] for d in data]
y = [d[-1] for d in data]

clf = DecisionTreeClassifier(
    criterion="gini",
    max_depth=3,
    random_state=42
)

clf.fit(X, y)
y_pred = clf.predict(X)

for actual, pred in zip(y, y_pred):
    print(f"Predicted: {pred} | Actual: {actual}")

print("\nAccuracy:", accuracy_score(y, y_pred))

Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 1 | Actual: 1
Predicted: 1 | Actual: 1
Predicted: 1 | Actual: 1
Predicted: 1 | Actual: 1
Predicted: 0 | Actual: 0

Accuracy: 1.0
